In [1]:
import os
import re
import warnings

import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")


matplotlib.rcParams["axes.unicode_minus"] = False  # 그래프 '-' 기호 표시

In [2]:
# 제공 데이터
c_permit = pd.read_csv("건축허가현황.csv")  # Construction_Permision
order = pd.read_csv("국내건설수주액.csv")  # Domestic construction Order Amount
unsold = pd.read_csv("미분양주택현황.csv")  # Unsold housing status
apt_price = pd.read_csv("아파트 실거래가격지수.csv")  # Actual transaction price index of apartments
monthly = pd.read_csv("유형별_주택월세통합가격지수.csv")  # House Monthly Rent Integrated Price Index
h_permit = pd.read_csv("주택건설인허가실적.csv")  # Housing construction permission result
house_price = pd.read_csv("주택매매가격지수(KB).csv")  # Housing Price Index
lease_price = pd.read_csv("주택전세가격지수(KB).csv")  # Housing Lease Price Index
land_price = pd.read_csv("지역별_지가변동률.csv")  # Land Price Variation Rate

In [3]:
df_list = [
    c_permit,
    order,
    unsold,
    apt_price,
    monthly,
    h_permit,
    house_price,
    lease_price,
    land_price,
]

In [4]:
# 날짜를 datetime형식으로 바꾼 뒤 index로 설정하는 함수적용
def date(df: pd.DataFrame) -> pd.DataFrame:
    df["Unnamed: 0"] = pd.to_datetime(df["Unnamed: 0"], format="%Y-%m-%d")
    df.rename(columns={"Unnamed: 0": "날짜"}, inplace=True)
    df.set_index("날짜", inplace=True)
    return df


for df in df_list:
    date(df)

c_permit.head()

,연면적-철근·철골조[㎡],연면적-조적조[㎡],연면적-목조[㎡],연면적-기타(자재별)[㎡],연면적-주거용[㎡],연면적-상업용[㎡],연면적-공업용[㎡],연면적-교육및사회용[㎡],연면적-기타(용도별)[㎡],연면적-자재별[㎡],...,동수-조적조[동수],동수-목조[동수],동수-기타(자재별)[동수],동수-주거용[동수],동수-상업용[동수],동수-공업용[동수],동수-교육및사회용[동수],동수-기타(용도별)[동수],동수-자재별[동수],동수-용도별[동수]
날짜,,,,,,,,,,,,,,,,,,,,,
2000-08-01,3898712.0,122063.0,5229.0,287185.0,1812355.0,1088979.0,687577.0,0.0,724278.0,4313189.0,...,1216.0,58.0,1078.0,2905.0,2008.0,741.0,0.0,1017.0,6671.0,6671.0
2000-09-01,3055754.0,113586.0,5918.0,236449.0,1273430.0,1014900.0,702592.0,0.0,420785.0,3411707.0,...,1128.0,70.0,793.0,2653.0,1756.0,652.0,0.0,840.0,5901.0,5901.0
2000-10-01,4104565.0,134648.0,8006.0,314620.0,1908535.0,1095874.0,1011477.0,0.0,545953.0,4561839.0,...,1290.0,61.0,1183.0,3246.0,2264.0,874.0,0.0,1120.0,7504.0,7504.0
2000-11-01,4337103.0,108212.0,7727.0,283128.0,2415543.0,1164943.0,613744.0,0.0,541940.0,4736170.0,...,1128.0,62.0,1088.0,2801.0,2050.0,816.0,0.0,1108.0,6775.0,6775.0
2000-12-01,6027008.0,161879.0,26079.0,453178.0,3450325.0,1412322.0,1069299.0,0.0,736198.0,6668144.0,...,941.0,70.0,986.0,2080.0,1619.0,748.0,0.0,1131.0,5578.0,5578.0


In [5]:
#시각화 시도

fig = px.line(c_permit, title="건축허가현황")
iplot(fig)

In [6]:
#시각화 시도2

fig = px.line(h_permit, title="주택건설인허가실적")
iplot(fig)

In [7]:
#제공된 데이터 분석으로만은 어떠한 결과를 가져오기 힘드므로 가설을 세운다
#최근 2030대 들이 스스로 서울 집한채 마련하기 어렵다는 말들을 확인해보자. 
#따라서 내 가설은 서울에는 주택공급이 적고, 수요가 많으며, 매매가격이 높을 것이다.

In [8]:
#미분양 주택은 서울이 적을 것이다. 공급이 적고 수요가 많기때문.


fig = px.line(unsold, title="미분양주택현황")
iplot(fig)

In [9]:
#수도권만 시각화 진행
fig = px.line(unsold[["전국[호]", "서울[호]", "수도권[호]"]], title="수도권, 서울, 전국 비교")
iplot(fig)

In [10]:
#최근 서울에는 미분양 아파트가 없음을 알 수 있다.
fig = px.line(unsold["서울[호]"], title="서울 미분양 주택 현황")
iplot(fig)

In [11]:
#그렇다면 아파트 가격은?
#아파트 실거래 가격지수 시각화

fig = px.line(apt_price, title="제공데이터 중 가격관련 데이터 분석")
iplot(fig)

In [12]:
#데이터에서 제공하는 기준점 활용

asc_price = apt_price.loc["2020-01"] > 100
asc_columns = asc_price[(asc_price[asc_price])].T.dropna().index.tolist()
asc_columns

['도심권[2017.11=100]',
 '동북권[2017.11=100]',
 '동남권[2017.11=100]',
 '대구[2017.11=100]',
 '인천[2017.11=100]',
 '광주[2017.11=100]',
 '대전[2017.11=100]',
 '세종[2017.11=100]',
 '경기[2017.11=100]',
 '서북권[2017.11=100]',
 '서남권[2017.11=100]',
 '전남[2017.11=100]',
 '지방광역시[2017.11=100]',
 '전국[2017.11=100]',
 '서울[2017.11=100]',
 '수도권[2017.11=100]']

In [13]:
dec_price = apt_price.loc["2020-01"] < 100
dec_columns = dec_price[(dec_price[dec_price])].T.dropna().index.tolist()
dec_columns

['부산[2017.11=100]',
 '울산[2017.11=100]',
 '강원[2017.11=100]',
 '충북[2017.11=100]',
 '충남[2017.11=100]',
 '전북[2017.11=100]',
 '경북[2017.11=100]',
 '경남[2017.11=100]',
 '제주[2017.11=100]',
 '지방도[2017.11=100]',
 '지방[2017.11=100]']

In [14]:
fig = px.line(apt_price[asc_columns], title="기준점 이후 상승세인 지역들")
iplot(fig)

In [15]:
fig = px.line(apt_price[dec_columns], title="기준점 이후 하락세인 지역들")
iplot(fig)

In [16]:
#기준점 이후 아파트 실거래 매매가격 격차가 큰폭만 뽑아보았다.

gap_price = (apt_price.loc["2021-01"][asc_columns].values) - (apt_price.loc["2018-01"][asc_columns].values)
gap_df = pd.DataFrame(data=gap_price, columns=asc_columns)
gap_df = gap_df.T.sort_values(by=0, ascending=False)
top_5 = gap_df.iloc[:5]
gap_top5 = top_5.index.tolist()
gap_top5

['세종[2017.11=100]',
 '동북권[2017.11=100]',
 '서울[2017.11=100]',
 '서북권[2017.11=100]',
 '서남권[2017.11=100]']

In [17]:
fig = px.line(apt_price['2018':][gap_top5], title="격차가 큰 상위 5개지역")
iplot(fig)

In [18]:
# 전국은 데이터가 너무 방대하고 필자의 지식이 부족하므로 범위를 서울로 좁혀보였다.
# 따라서 서울을 기준으로 전국과 비교해보자

d_seoul = apt_price[['서울[2017.11=100]','전국[2017.11=100]']]
fig = px.line(d_seoul, title="전국과 서울 아파트 실거래가 비교")
iplot(fig)

In [19]:
# 해당 지역만을 리스트로 가져오는 함수
def location(name):
    name = [
        "종합-" + name + "[2017.11=100]",
        "아파트-" + name + "[2017.11=100]",
        "연립다세대-" + name + "[2017.11=100]",
        "단독주택-" + name + "[2017.11=100]",
    ]
    return name


location("서울")

['종합-서울[2017.11=100]',
 '아파트-서울[2017.11=100]',
 '연립다세대-서울[2017.11=100]',
 '단독주택-서울[2017.11=100]']

In [20]:
fig = px.line(monthly[location("전국")], title="주택월세통합가격지수 - 전국")
iplot(fig)

In [21]:
fig = px.line(monthly[location("서울")], title="주택월세통합가격지수 - 서울")
iplot(fig)

In [22]:
#전국과 서울 월세 변화 비교 -> 전국에 비해 서울은 하락폭도 작고 상승도 어마무시하다.
# 또한 전국 데이터에 서울 데이터도 포함되어 있을테니 서울을 제외한다면 전국과 서울 간격이 더욱 클 것이다.
d_monthly = monthly[["아파트-전국[2017.11=100]", "아파트-서울[2017.11=100]"]]
fig = px.line(d_monthly, title="전국과 서울의 아파트 월세 변화")
iplot(fig)

In [23]:
# 전국과 서울 주택 매매가 비교

house_price.head(3)

,총지수[2019.01=100],단독주택[2019.01=100],연립주택[2019.01=100],아파트[2019.01=100],아파트(서울)[2019.01=100],총지수(서울)[2019.01=100]
날짜,,,,,,
1986-01-01,34.656,61.347,43.053,20.973,17.153,30.044
1986-02-01,34.656,61.347,42.925,20.973,17.183,30.044
1986-03-01,34.708,61.513,42.733,20.935,17.153,30.002


In [24]:
fig = px.line(house_price, title="주택매매가격지수")
iplot(fig)

In [25]:
#1990년대는 너무 과거라 최근 5년으로 범위를 좁혀보았다.

fig = px.line(
    house_price.loc[
        "2014":,
        [
            "총지수[2019.01=100]",
            "아파트[2019.01=100]",
            "아파트(서울)[2019.01=100]",
            "총지수(서울)[2019.01=100]",
        ],
    ],
    title="2019 5년전후의 총 지수와 아파트 매매가 최근 동향 비교",
)
iplot(fig)

In [26]:
# 데이터에서 제공하는 2019년도를 기점으로 분석.

fig = px.line(
    house_price.loc["2019":, ["아파트[2019.01=100]", "아파트(서울)[2019.01=100]"]],
    title="2019 이후 전국과 서울 아파트 최근 매매가 동향 ",
)
iplot(fig)

In [27]:
#주택 전세가격지수도 관찰

lease_price.head(3)

,총지수[2019.01=100],단독주택[2019.01=100],연립주택[2019.01=100],아파트[2019.01=100],아파트(서울)[2019.01=100],총지수(서울)[2019.01=100]
날짜,,,,,,
1986-01-01,17.601,35.506,18.059,10.793,9.782,16.755
1986-02-01,17.853,35.731,18.155,11.104,10.195,17.066
1986-03-01,18.273,36.553,18.823,11.446,10.550,17.573


In [28]:
fig = px.line(lease_price, title="주택전세가격지수")
iplot(fig)

In [29]:
# 아파트만 비교 -> 최근 서울 집값 상승이 전국에 비해서도 무섭다.

fig = px.line(
    lease_price.loc[
        "2014":,
        [
            "총지수[2019.01=100]",
            "아파트[2019.01=100]",
            "아파트(서울)[2019.01=100]",
            "총지수(서울)[2019.01=100]",
        ],
    ],
    title="2019 5년전후의 총지수와 아파트 전세가 최근 동향 비교",
)
iplot(fig)

In [30]:
fig = px.line(
    lease_price.loc["2019":, ["아파트[2019.01=100]", "아파트(서울)[2019.01=100]"]],
    title="2019년 이후 전국과 서울 아파트 최근 전세가 동향 ",
)

iplot(fig)

In [31]:
#월세의 변화폭은 집값에 비해 미미하기 때문에 작게 관찰.

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=d_seoul.index,
        y=d_seoul["서울[2017.11=100]"],
        mode="lines",
        name="아파트 실거래가",
    )
)

fig.add_trace(
    go.Scatter(
        x=d_monthly.index,
        y=d_monthly["아파트-서울[2017.11=100]"],
        mode="lines",
        name="아파트 월세가",
    )
)

fig.add_trace(
    go.Scatter(
        x=house_price.loc["2006":, :].index,
        y=house_price.loc["2006":, "아파트(서울)[2019.01=100]"],
        mode="lines",
        name="아파트 매매가",
    )
)

fig.add_trace(
    go.Scatter(
        x=lease_price.loc["2006":, :].index,
        y=lease_price.loc["2006":, "아파트(서울)[2019.01=100]"],
        mode="lines",
        name="아파트 전세가",
    )
)

fig.update_layout(title="<b>서울시 아파트 가격에 대한 가격자료</b>")
iplot(fig)

In [32]:
# 정리하자면 
# 가설 : 최근 2030대가 서울에 집 한채 마련하기 어렵다는 말이 나온다.
# 이는 서울 주택의 공급이 적고, 수요가 많아서 매매가격이 치솟는 것이다.
# 진행 : 미분양 주택을 통해 공급과 수요의 갭차이를 알아보고, 전국과 서울의 집값상승률을 분석해본다.
# 이를 통해 상대적으로도 서울의 상승률이 어마무시하다는걸 알 수 있으며
# 전국과 서울의 비교보다 좀 더 자세하게, 서울시 내에서 집값의 차이는 클까?
# 결론 : 미분양 주택은 0에 수렴하여 수요는 이미 공급을 만족했고, 집값 상승으로 미루어보아 수요가 공급을 초과함을 알 수 있다.
# 전국에 비해 서울의 집값 상승률이 월등히 높았고, 이는 서울로의 수요가 전국에 비해 훨씬 많다는걸 알 수 있었다.
# 계속되는 서울로의 주택 수요에 따라 2030대는 집을 마련할 기회를 얻기도 전에 집값이 폭등한 상태이므로 마련하기 힘들 것 이다.

# 추가 진행 : 그래도 서울에 넘쳐나는 수요에 따라 서울시 내에서의 집값 차이는 어떠한지 알아보자.

In [33]:
# 서울지역 행정구역별 아파트매매가격지수 분석
# 데이터 -> kosis 아파트매매 실거래가격지수 데이터 전처리 및 가공

seouls = pd.read_excel("서울지역별_아파트매매가격지수.xlsx")
seouls.head()

,행정구역별(1),행정구역별(2),2017,2018,2019,2019.75,2020,2021
0,서울,종로구,93.3,100,115.6,120.4,124.6,141.8
1,서울,중구,91.4,100,125.3,127.7,132.6,159.1
2,서울,용산구,87.9,100,124.7,130.8,137.6,156.4
3,서울,성동구,86.5,100,127.9,130.6,138.1,174.3
4,서울,광진구,89.3,100,121.3,124.2,134.5,164.0


In [34]:
seouls.columns

Index(['행정구역별(1)', '행정구역별(2)', 2017, 2018, 2019, 2019.75, 2020, 2021], dtype='object')

In [35]:
seouls.drop(2019.75, axis=1, inplace=True)
seouls.head()

,행정구역별(1),행정구역별(2),2017,2018,2019,2020,2021
0,서울,종로구,93.3,100,115.6,124.6,141.8
1,서울,중구,91.4,100,125.3,132.6,159.1
2,서울,용산구,87.9,100,124.7,137.6,156.4
3,서울,성동구,86.5,100,127.9,138.1,174.3
4,서울,광진구,89.3,100,121.3,134.5,164.0


In [36]:
seouls["평균가격지수"] = seouls[[2017, 2018, 2019, 2020, 2021]].mean(axis=1)
seouls.head()

,행정구역별(1),행정구역별(2),2017,2018,2019,2020,2021,평균가격지수
0,서울,종로구,93.3,100,115.6,124.6,141.8,115.06
1,서울,중구,91.4,100,125.3,132.6,159.1,121.68
2,서울,용산구,87.9,100,124.7,137.6,156.4,121.32
3,서울,성동구,86.5,100,127.9,138.1,174.3,125.36
4,서울,광진구,89.3,100,121.3,134.5,164.0,121.82


In [37]:
# 다양한 시각화 기법 시도 -> folium을 이용한 시각화 진행
import folium
from folium import plugins

In [38]:
def draw_data(x, color):
    # 서울시 행정구역 json 불러오기
    state_geo = "https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json"
    m = folium.Map(
        location=[37.562225, 126.978555],
        width="100%",
        height="100%",
        zoom_start=11,
    )
    choropleth = folium.Choropleth(
        geo_data=state_geo,
#         name=x,
        data=seouls,
        columns=["행정구역별(2)", x],
        key_on="feature.properties.name",
        fill_color=color,
        fill_opacity=0.9,
        line_opacity=0.6,
        legend=True,
        legend_name=x,
    ).add_to(m)

    
    return m

In [39]:
# 지도시각화
draw_data("평균가격지수", "Blues")